# Рыночные признаки

- Бета коэффициент к индексу IMOEX
- Корреляция за 60 дней
- Волатильность индекса

Оценивается связь тикера с общерыночными факторами.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


## Функции расчета рыночных признаков


In [ ]:
def calculate_beta(stock_returns, market_returns, window=60):
    """Расчет бета коэффициента к рынку (индексу)"""
    beta_list = []
    
    for i in range(len(stock_returns)):
        if i < window:
            beta_list.append(np.nan)
            continue
        
        stock_window = stock_returns.iloc[i-window:i]
        market_window = market_returns.iloc[i-window:i]
        
        # Коэффициент корреляции * (std_stock / std_market)
        covariance = stock_window.cov(market_window)
        market_variance = market_window.var()
        
        beta = covariance / market_variance if market_variance != 0 else 0
        beta_list.append(beta)
    
    return pd.Series(beta_list, index=stock_returns.index)


def rolling_correlation(stock_returns, market_returns, window=60):
    """Скользящая корреляция с индексом"""
    return stock_returns.rolling(window=window).corr(market_returns)


def market_volatility(market_returns, window=30):
    """Волатильность рынка"""
    return market_returns.rolling(window=window).std() * np.sqrt(252)

print("✅ Функции рыночных признаков загружены")


## Загрузка данных тикера и индекса


In [ ]:
# Загрузка данных тикера
DATA_DIR = Path('data') / 'processed'
ticker = 'SBER'

try:
    df = pd.read_parquet(DATA_DIR / f"{ticker}_ohlcv_returns.parquet")
    print(f"✅ Тикер {ticker} загружен: {len(df)} записей")
except FileNotFoundError:
    print(f"❌ Файл для {ticker} не найден")
    df = None

# Загрузка данных индекса IMOEX
try:
    index_df = pd.read_parquet(DATA_DIR / "IMOEX_ohlcv_returns.parquet")
    print(f"✅ Индекс IMOEX загружен: {len(index_df)} записей")
    
    if df is not None:
        # Объединяем по датам
        merged = pd.merge(df, index_df[['date', 'log_return']], on='date', how='left', suffixes=('', '_index'))
        
        # Переименовываем для ясности
        merged.rename(columns={'log_return_index': 'index_return'}, inplace=True)
        df = merged
        
        # Проверяем сколько данных совпало
        valid_rows = df['index_return'].notna().sum()
        print(f"✅ Объединено: {valid_rows} совпадающих дат из {len(df)}")
    
except FileNotFoundError:
    print("⚠️ Индекс IMOEX не найден в processed. Используем синтетический индекс")
    if df is not None:
        # Создаем синтетический индекс для демонстрации
        df['index_return'] = df['log_return'] * 0.7 + np.random.randn(len(df)) * 0.01

if df is not None:
    print(f"✅ Данные подготовлены: {len(df)} записей для анализа")
else:
    print("❌ Не удалось загрузить данные")


## Расчет рыночных признаков


In [ ]:
# Расчет бета коэффициента
df['beta_60'] = calculate_beta(df['log_return'], df['index_return'], window=60)

# Корреляция с индексом
df['correlation_60'] = rolling_correlation(df['log_return'], df['index_return'], window=60)

# Волатильность индекса
df['index_vol_30'] = market_volatility(df['index_return'], window=30)

print("✅ Рыночные признаки рассчитаны")
print(f"\nПример данных:")
print(df[['date', 'close', 'log_return', 'beta_60', 'correlation_60', 'index_vol_30']].tail())


## Сохранение результатов


In [ ]:
OUTPUT_DIR = Path('data') / 'features'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

output_path = OUTPUT_DIR / f"{ticker}_market_features.parquet"
df.to_parquet(output_path, index=False)
print(f"✅ Сохранено: {output_path}")
